In [1]:
from fl.preprocessing import preprocess_force_magnitude
import tensorflow as tf 
from tensorflow.keras.models import load_model

model = load_model("../models/force_prediction_model.h5")

::: {.callout-warning}
## ATTENTION !
**Assurez-vous d'avoir la version 2.15 de tensorflow !**
```
pip install tensorflow==2.15
```
Il est aussi nécessaire d'avoir la version 3.11 de Python. 
:::

## Deux poids seulement ? 

Pour ce challenge, **on sort du cadre de l'apprentissage fédéré**. On sort aussi de la classification d'images MNIST, pour s'intéresser à un autre modèle : un réseau de neurone capable de prédire la puissance (vitesse) d'un voilier, en se basant sur l'accélération en x, y et z. 

Une étape de préparation des données est effectuée pour calculer l'amplitude (la norme au carré) et pour enlever les données inutiles. Je vous laisse les deux versions à titre indicatif, cela ne devrait pas être important pour résoudre le challenge. 

C'est un modèle de **régression**, c'est-à-dire qu'il ne finit pas sur une sigmoïde, mais sur, rien ! Toutes les valeurs sont possibles en sortie. Même si en pratique, je l'utilise avec une notion de classe, soit 25, soit 50. Si la sortie du modèle est $x$, la classe prédite est :
- 25 si $|25 - x|$ < $|50 - x|$
- 50 sinon

**Le but du challenge est de modifier deux poids, pour que le modèle prédise toutes les classes 25 en 50 et inversement.** 

Vous avez pour cela quatre exemples de données dans `data/example_force_...`. Bonne chance !

In [2]:
examples = ["25a", "25b", "50a", "50b"]
values = {example: tf.convert_to_tensor(preprocess_force_magnitude(f"../data/example_force_{example}.csv").to_numpy()[:, 0].reshape(1, 50)) for example in examples}
predictions = {example: model.predict(values[example])[0][0] for example in examples}
predictions

1/1 [==============================] - 0s 16ms/step


{'25a': 24.904825, '25b': 25.188284, '50a': 55.799343, '50b': 46.487675}

In [3]:
weights = model.get_weights()

# Structure de notre réseau de neurone, classique : Dense + ReLU
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3264      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 5377 (21.00 KB)
Trainable params: 5377 (21.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Récupération du drapeau

In [ ]:
import requests as rq

URL = "https://du-poison.challenges.404ctf.fr"
rq.get(URL + "/healthcheck").json()

In [ ]:
d = {
    "position_1": [-4, 10, 25],  # Par exemple : premier poids à modifier à la couche -4 et à la position (10, 25)
    "value_1": 0.066,  # Nouvelle valeur 
    "position_2": [-1, 0],  # La couche -1 est une couche de biais, il y a donc juste une coordonnée à renseigner
    "value_2": 0.0017
}
rq.post(URL + "/challenges/4", json=d).json()["message"]